In [1]:
import fireworks.client
import os
import dotenv
import chromadb
import json
from tqdm.auto import tqdm
import pandas as pd
import random

dotenv.load_dotenv()

fireworks.client.api_key = os.getenv("FIREWORKS_API_KEY")

/home/grace-nyutu/Documents/Precision_RAG/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_completion(prompt, model=None, max_tokens=60):

    fw_model_dir = "accounts/fireworks/models/"

    if model is None:
        model = fw_model_dir + "llama-v2-7b"
    else:
        model = fw_model_dir + model

    completion = fireworks.client.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=0
    )

    return completion.choices[0].text

In [3]:
mistral_llm = "mistral-7b-instruct-4k"

In [4]:
prompt = """[INST]
Given the following wedding guest data, write a very short 3-sentences thank you letter:

{
  "name": "Grace Nyutu",
  "relationship": "Bride's best friend",
  "hometown": "Luton, LU",
  "fun_fact": "Went to the moon in 2020",
  "attending_with": "Rachel White",
  "bride_groom_name": "Solomon"
}

Use only the data provided in the JSON object above.

The senders of the letter is the bride and groom, Solomon and Mary.
[/INST]"""

get_completion(prompt, model=mistral_llm, max_tokens=150)

' Dear Grace,\n\nWe wanted to take a moment to thank you for attending our wedding. It was such a pleasure to have you there as our guest, and we are grateful for your presence on this special day. We also wanted to share with you that we heard about your incredible experience of going to the moon in 2020, and we are truly impressed by your achievement. We hope you had a wonderful time at our wedding and that you continue to have many more adventures in the future.\n\nSincerely,\nSolomon and Mary'

In [6]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        batch_embeddings = embedding_model.encode(input)
        return batch_embeddings.tolist()

embed_fn = MyEmbeddingFunction()

# Initialize the chromadb directory, and client.
client = chromadb.PersistentClient(path="./chromadb")

# create collection
collection = client.get_or_create_collection(
    name=f"challenge_docs"
)

/home/grace-nyutu/Documents/Precision_RAG/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
# Generate embeddings, and index titles in batches
batch_size = 50

# loop through batches and generated + store embeddings
for i in tqdm(range(0, len(challenge_docs), batch_size)):

    i_end = min(i + batch_size, len(challenge_docs))
    batch = challenge_docs[i : i + batch_size]

    batch_ids = [str(sum(ord(c) + random.randint(1, 10000) for c in paper["Paragraph"])) for paper in batch]

    # generate embeddings
    batch_embeddings = embedding_model.encode(batch)

    # upsert to chromadb
    collection.upsert(
        ids=batch_ids,
        embeddings=batch_embeddings.tolist(),
    )

NameError: name 'challenge_docs' is not defined

Chunking

In [ ]:
from langchain.document_loaders import ReadTheDocsLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Text splitter
chunk_size = 300
chunk_overlap = 50
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
)

# Chunk a sample section
chunks = text_splitter.create_documents(
    texts=[week1_dict["text"]], 
    metadatas=[{"source": week1_dict["source"]}])
print (chunks[0])

TypeError: list indices must be integers or slices, not str

In [ ]:
def chunk_section(section, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " ", ""],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len)
    chunks = text_splitter.create_documents(
        texts=[sample_section["text"]], 
        metadatas=[{"source": sample_section["source"]}])
    return [{"text": chunk.page_content, "source": chunk.metadata["source"]} for chunk in chunks]

# Scale chunking
chunks_ds = sections_ds.flat_map(partial(
    chunk_section, 
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap))
print(f"{chunks_ds.count()} chunks")
chunks_ds.show(1)
